In [1]:
import pandas as pd
import glob
import math
from itertools import tee

In [2]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [3]:
# Formel: distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
# Theoretische maximale Distanz: 126 meter

def distance(row1, row2):
    distance = math.sqrt((row1['pos_x'] - row2['pos_x']) ** 2 +(row1['pos_z'] - row2['pos_z']) ** 2)
    if(distance > 1):
        distance = 0
    return distance

In [4]:
all_files = glob.glob("data" + "/*.csv")

In [5]:
ids = []
for filename in all_files:
    id = filename.split('_')
    ids.append(id[2].replace('study', 'p'))    

In [6]:
datasets = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=False, header=0, sep=';')
    datasets.append(df)

In [7]:
columns = ['Id','Distance']
df_distances_participants = pd.DataFrame(columns=columns)

In [8]:
#rausrechnen das Teleport springt...
j = 0
for i in range(len(datasets)):
    dist = 0;
    df = datasets[i]
    datasets_player = []
    for i, row in df.iterrows():
        if row['gameobject_name'] == 'Player':
            datasets_player.append(row)
    df_player = pd.DataFrame(datasets_player)
    
    for (i1, row1), (i2, row2) in pairwise(df_player.iterrows()):
        dist += distance(row1,row2)
    df_distances_participants = df_distances_participants.append({'Id':ids[j],'Distance':round(dist,2)}, ignore_index=True)
    j += 1

In [9]:
df_distances_participants

,Id,Distance
0,p13,109.65
1,p14,102.64
2,p15,99.54
3,p16,109.59
4,p17,105.18
5,p18,119.26
6,p19,93.49
7,p20,100.78
8,p21,106.97
9,p22,99.08


In [12]:
df_distances_participants.describe()

,Distance
count,12.000000
mean,104.568333
std,6.731126
min,93.490000
25%,100.267500
50%,103.910000
75%,108.495000
max,119.260000


In [11]:
# save dataframe
df_distances_participants.to_csv('CalculationWalkedDistances_SDE.csv')

# ... and describe
df_distances_participants.describe().to_csv('Describe_WalkedDistances_SDE.csv')